# MLflow Validation Test - Self-Contained Model Registration

This notebook generates synthetic data and registers a simple model to validate our MLflow infrastructure.

In [ ]:
import numpy as np
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import os

print("Libraries imported successfully")
print(f"MLflow tracking URI: {mlflow.get_tracking_uri()}")

In [ ]:
# Generate realistic sensor data matching our database schema
np.random.seed(42)

# Define realistic sensor characteristics based on our database
sensor_types = {
    'vibration': {'min': 28, 'max': 82, 'avg': 60, 'unit': 'mm/s'},
    'pressure': {'min': 29, 'max': 55, 'avg': 46, 'unit': 'kPa'},
    'temperature': {'min': 11, 'max': 89, 'avg': 53, 'unit': 'C'},
    'humidity': {'min': 32, 'max': 88, 'avg': 66, 'unit': '%'},
    'voltage': {'min': 21, 'max': 58, 'avg': 43, 'unit': 'V'}
}

n_samples = 1000
sensor_features = []
feature_names = []

# Generate realistic features for each sensor type
for sensor_type, params in sensor_types.items():
    # Normal operation: centered around average with some variance
    normal_std = (params['max'] - params['min']) * 0.15  # 15% of range as std
    normal_data = np.random.normal(params['avg'], normal_std, int(n_samples * 0.9))
    
    # Anomalous data: values near limits or outside normal range
    anomaly_low = np.random.uniform(params['min'], params['avg'] - 2*normal_std, int(n_samples * 0.05))
    anomaly_high = np.random.uniform(params['avg'] + 2*normal_std, params['max'], int(n_samples * 0.05))
    
    # Combine normal and anomalous data
    sensor_data = np.concatenate([normal_data, anomaly_low, anomaly_high])
    
    # Clip to realistic bounds
    sensor_data = np.clip(sensor_data, params['min'], params['max'])
    sensor_features.append(sensor_data)
    feature_names.append(f"{sensor_type}_{params['unit']}")

# Stack features horizontally
X_synthetic = np.column_stack(sensor_features)

# Create labels: 0 for normal (first 90%), 1 for anomalous (last 10%)
y_synthetic = np.concatenate([
    np.zeros(int(n_samples * 0.9)),  # Normal
    np.ones(int(n_samples * 0.1))    # Anomalous
])

# Shuffle the data to mix normal and anomalous samples
shuffle_idx = np.random.permutation(len(X_synthetic))
X_synthetic = X_synthetic[shuffle_idx]
y_synthetic = y_synthetic[shuffle_idx]

print(f"Generated {len(X_synthetic)} realistic sensor samples")
print(f"Features: {feature_names}")
print(f"Normal samples: {(y_synthetic == 0).sum()}, Anomalous samples: {(y_synthetic == 1).sum()}")
print("\nSample ranges:")
for i, (name, sensor_type) in enumerate(zip(feature_names, sensor_types.keys())):
    print(f"  {name}: {X_synthetic[:, i].min():.1f} - {X_synthetic[:, i].max():.1f} (target: {sensor_types[sensor_type]['min']}-{sensor_types[sensor_type]['max']})")

In [ ]:
# Configure MLflow
mlflow.set_tracking_uri("http://mlflow:5000")
mlflow.set_experiment("Synthetic_Data_Validation")

print(f"MLflow configured to: {mlflow.get_tracking_uri()}")
print(f"Active experiment: {mlflow.get_experiment_by_name('Synthetic_Data_Validation')}")

In [ ]:
# Split data for training and testing
X_train, X_test, y_train, y_test = train_test_split(
    X_synthetic, y_synthetic, test_size=0.3, random_state=42, stratify=y_synthetic
)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Training set: {X_train_scaled.shape}")
print(f"Test set: {X_test_scaled.shape}")

In [ ]:
# Train and register model with MLflow
with mlflow.start_run(run_name="Realistic_Sensor_Validation_IsolationForest") as run:
    # Log parameters
    contamination = 0.1
    mlflow.log_param("model_type", "IsolationForest")
    mlflow.log_param("contamination", contamination)
    mlflow.log_param("n_samples", n_samples)
    mlflow.log_param("n_features", len(feature_names))
    mlflow.log_param("data_type", "realistic_synthetic")
    mlflow.log_param("sensor_types", list(sensor_types.keys()))
    
    # Train model
    model = IsolationForest(contamination=contamination, random_state=42)
    model.fit(X_train_scaled)
    
    # Make predictions
    y_pred = model.predict(X_test_scaled)
    y_pred_binary = (y_pred == -1).astype(int)  # Convert to binary (1 for anomaly)
    
    # Calculate comprehensive metrics
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
    
    accuracy = accuracy_score(y_test, y_pred_binary)
    precision = precision_score(y_test, y_pred_binary, zero_division=0)
    recall = recall_score(y_test, y_pred_binary, zero_division=0)
    f1 = f1_score(y_test, y_pred_binary, zero_division=0)
    
    # Log metrics
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)
    
    # Log feature names and sensor types as artifacts
    with open("/tmp/feature_names_realistic.txt", "w") as f:
        f.write("\n".join(feature_names))
    mlflow.log_artifact("/tmp/feature_names_realistic.txt")
    
    with open("/tmp/sensor_config.txt", "w") as f:
        for sensor_type, params in sensor_types.items():
            f.write(f"{sensor_type}: {params['min']}-{params['max']} {params['unit']} (avg: {params['avg']})\n")
    mlflow.log_artifact("/tmp/sensor_config.txt")
    
    # Register the model
    model_name = "realistic_sensor_validation_isolation_forest"
    mlflow.sklearn.log_model(
        model, 
        "model",
        registered_model_name=model_name
    )
    
    print(f"Model registered as: {model_name}")
    print(f"Run ID: {run.info.run_id}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")
    
    # Print classification report
    print("\nDetailed Classification Report:")
    print(classification_report(y_test, y_pred_binary, target_names=['Normal', 'Anomaly']))

In [ ]:
# Test model loading from MLflow
print("\n=== Testing Model Loading ===")
try:
    # Load the model back from MLflow
    loaded_model = mlflow.sklearn.load_model(f"models:/{model_name}/latest")
    
    # Test prediction with the loaded model
    test_sample = X_test_scaled[:5]  # Test with first 5 samples
    predictions = loaded_model.predict(test_sample)
    
    print(f"Successfully loaded model: {model_name}")
    print(f"Test predictions: {predictions}")
    print(f"Actual labels: {y_test[:5]}")
    
    # Test with a realistic sensor reading
    realistic_sample = np.array([[60.0, 46.0, 53.0, 66.0, 43.0]])  # Normal values
    realistic_scaled = scaler.transform(realistic_sample)
    realistic_pred = loaded_model.predict(realistic_scaled)
    print(f"Normal sensor reading prediction: {realistic_pred[0]} (should be 1 for normal)")
    
    print("✅ MLflow model loading validation PASSED")
    
except Exception as e:
    print(f"❌ MLflow model loading validation FAILED: {e}")
    raise

In [ ]:
print("\n=== Synthetic Data Validation Summary ===")
print("✅ Synthetic data generation: SUCCESS")
print("✅ Model training: SUCCESS")
print("✅ MLflow logging: SUCCESS")
print("✅ Model registration: SUCCESS")
print("✅ Model loading validation: SUCCESS")
print("\n🎉 All validation tests passed! MLflow infrastructure is working correctly.")